In [1]:
!pip install -qU langchain chromadb langchainhub bs4 gradio sentence-transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
google-cloud-pubsublite 1.8.3 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.4.0 which is incompatible.
jupyterlab 4.0.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.0 requires jupyterlab<5.0.0a0,>=4.0.6, but you have jupyterlab 4.0.5 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.0.1 requires kubernetes<27,>=8.0.0, but you have kubernetes 28.1.0 which 

In [2]:
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = user_secrets.get_secret("HF_READ_TOKEN")

In [3]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [4]:
loader = WebBaseLoader(
    web_path=('http://karpathy.github.io/2019/04/25/recipe/',),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_=('post-content', 'post-title', 'post-header'))
    ),
)
docs = loader.load()

In [5]:
len(docs)

1

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [7]:
len(splits)

34

In [8]:
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever = vectorstore.as_retriever()

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
prompt = hub.pull("rlm/rag-prompt")
llm = HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta', model_kwargs={"temperature": 0.5, "max_length": 4096})

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [11]:
llm

HuggingFaceHub(client=InferenceAPI(api_url='https://api-inference.huggingface.co/pipeline/text-generation/HuggingFaceH4/zephyr-7b-beta', task='text-generation', options={'wait_for_model': True, 'use_gpu': False}), repo_id='HuggingFaceH4/zephyr-7b-beta', model_kwargs={'temperature': 0.5, 'max_length': 4096})

In [12]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever)

In [13]:
chat_history = []

In [14]:
question = "What is becoming one with the data?"
result = qa({"question": question, "chat_history": chat_history})
chat_history.append((question, result["answer"]))
print(result["answer"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 Overfitting. When a model is so complex that it can memorize the training data, it is said to be overfitting. In this stage, the goal is to find a balance between complexity and generalization. This is achieved by regularization techniques that penalize complex models and encourage simpler ones. The optimal level of regularization depends on the problem and the dataset, and finding it requires iterating through different levels of regularization. The key is to ensure that the model can still generalize to unseen data, and not just memorize the training set. This is a critical stage in the model development process, as a model that overfits will not perform well on new, unseen data.


In [15]:
def load_article(path):
    loader = WebBaseLoader(
        web_path=(path,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(class_=('post-content', 'post-title', 'post-header'))
        ),
    )
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
    retriever = vectorstore.as_retriever()
    prompt = hub.pull("rlm/rag-prompt")
    llm = HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta', model_kwargs={"temperature": 0.5, "max_length": 4096})
    global qa
    qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever)
    return 'Ready!!!'

In [16]:
load_article(path='http://karpathy.github.io/2019/04/25/recipe/')

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


'Ready!!!'

In [17]:
def add_text(history, text):
    history = history + [(text, None)]
    return history, ''

def bot(history):
    response = infer(history[-1][0])
    history[-1][1] = response['result']
    return history

def infer(question):
    query = question
    result = qa({'query': query})
    return result

In [1]:
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            article = gr.Textbox(label='Article Path', placeholder='Enter the path')
            article_status = gr.Textbox(label='Status', placeholder='', interactive=False)
            load = gr.Button('Load Article...')
        
        chatbot = gr.Chatbot([], elem_id='Chatbot')
        question = gr.Textbox(label='Question', placeholder='Type your query')
        submit_btn = gr.Button('Submit')
    
    load.click(load_article, inputs=[article], outputs=[article_status], queue=False)
    question.submit(add_text, [chatbot, question], [chatbot, question]).then(bot, chatbot, chatbot)
    submit_btn.click(add_text, [chatbot, question], [chatbot, question]).then(bot, chatbot, chatbot)
    
demo.launch(share=True)

NameError: name 'gr' is not defined